<a href="https://colab.research.google.com/github/Dheeraj024/neural-network-bagging-/blob/main/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mnist_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mnist_train.csv")
mnist_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mnist_test.csv")

In [ ]:
mnist_train

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def sampling(tdata,val):
    len1= int(len(tdata)*val)
    t1 = tdata.sample(frac=1)
    val_data = t1.iloc[0:len1,:]
    train = t1.iloc[len1:,:]
    return val_data,train

def generator(train,batch_size):
    count = 0
    while count<len(train):
        data = train.iloc[count:count+batch_size,:]
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        yield X.values,Y.values
        count+=batch_size

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Initialize weights and biases
        self.W1 = np.random.randn(self.input_size, self.hidden_size) * 0.01
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.output_size) * 0.01
        self.b2 = np.zeros((1, self.output_size))

    def forward(self, X):
        # Forward pass
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.maximum(0, self.z1)  # ReLU activation
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = np.exp(self.z2 - np.max(self.z2, axis=1, keepdims=True))  # Softmax activation
        self.a2 /= np.sum(self.a2, axis=1, keepdims=True)
        return self.a2

    def backward(self, X, y, learning_rate):
        # Backward pass
        m = X.shape[0]

        dZ2 = self.a2
        dZ2[range(m), y] -= 1
        dZ2 /= m

        dW2 = np.dot(self.a1.T, dZ2)
        db2 = np.sum(dZ2, axis=0)

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * (self.a1 > 0)  # ReLU derivative

        dW1 = np.dot(X.T, dZ1)
        db1 = np.sum(dZ1, axis=0)

        # Update weights and biases
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

    def train(self, X, y, epochs, learning_rate):
        epsilon = 1e-8
        for epoch in range(epochs):
            # Forward pass
            a2 = self.forward(X)

            # Compute loss
            loss = -np.sum(np.log(a2[range(X.shape[0]), y]+epsilon)) / X.shape[0]

            # Backward pass
            self.backward(X, y, learning_rate)

            # Print loss
            # if (epoch+1) % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss:.4f}')

    def predict(self, X):
        # Predict class labels
        a2 = self.forward(X)
        y_pred = np.argmax(a2, axis=1)
        return y_pred

# Create and train the neural network

In [ ]:
x_train = mnist_train.iloc[:,1:]

In [ ]:
x_train

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train = mnist_train.iloc[:,0]
y_train.head()

0    5
1    0
2    4
3    1
4    9
Name: label, dtype: int64

In [ ]:
type(y_train)

pandas.core.series.Series

In [ ]:
x_test = mnist_test.iloc[:,1:]
y_test = mnist_test.iloc[:,0]

In [ ]:
nn = NeuralNetwork(x_train.shape[1],128,10)

In [ ]:
nn.train(x_train,y_train,30,0.001)

Epoch 0, Loss: 0.5168
Epoch 1, Loss: 0.5104
Epoch 2, Loss: 0.5043
Epoch 3, Loss: 0.4986
Epoch 4, Loss: 0.4932
Epoch 5, Loss: 0.4881
Epoch 6, Loss: 0.4832
Epoch 7, Loss: 0.4786
Epoch 8, Loss: 0.4741
Epoch 9, Loss: 0.4698
Epoch 10, Loss: 0.4657
Epoch 11, Loss: 0.4617
Epoch 12, Loss: 0.4579
Epoch 13, Loss: 0.4543
Epoch 14, Loss: 0.4507
Epoch 15, Loss: 0.4473
Epoch 16, Loss: 0.4439
Epoch 17, Loss: 0.4407
Epoch 18, Loss: 0.4376
Epoch 19, Loss: 0.4345
Epoch 20, Loss: 0.4316
Epoch 21, Loss: 0.4287
Epoch 22, Loss: 0.4259
Epoch 23, Loss: 0.4232
Epoch 24, Loss: 0.4206
Epoch 25, Loss: 0.4180
Epoch 26, Loss: 0.4155
Epoch 27, Loss: 0.4131
Epoch 28, Loss: 0.4107
Epoch 29, Loss: 0.4084


In [ ]:
print(y_test)

0       7
1       2
2       1
3       0
4       4
       ..
9995    2
9996    3
9997    4
9998    5
9999    6
Name: label, Length: 10000, dtype: int64


In [ ]:
prediction = nn.predict(x_test)

In [ ]:
prediction

array([7, 2, 1, ..., 4, 5, 6])

In [ ]:
model_net = []
for i in range(5):
    validation,train_data = sampling(mnist_train,0.2)
    v_data = validation.iloc[:,1:]
    v_label = validation.iloc[:,0]
    # gen = generator(train_data,8)
    # x,y = gen
    x_data = train_data.iloc[:,1:]
    y_data = train_data.iloc[:,0]
    nn = NeuralNetwork(x_train.shape[1],128,10)
    nn.train(x_data,y_data,50,0.001)
    model_net.append(nn)

Epoch 0, Loss: 3.3884
Epoch 1, Loss: 2.7784
Epoch 2, Loss: 2.0378
Epoch 3, Loss: 1.7384
Epoch 4, Loss: 1.4843
Epoch 5, Loss: 1.2964
Epoch 6, Loss: 1.2085
Epoch 7, Loss: 1.0772
Epoch 8, Loss: 1.0238
Epoch 9, Loss: 0.9473
Epoch 10, Loss: 0.9102
Epoch 11, Loss: 0.8612
Epoch 12, Loss: 0.8331
Epoch 13, Loss: 0.7966
Epoch 14, Loss: 0.7734
Epoch 15, Loss: 0.7444
Epoch 16, Loss: 0.7246
Epoch 17, Loss: 0.7008
Epoch 18, Loss: 0.6835
Epoch 19, Loss: 0.6635
Epoch 20, Loss: 0.6484
Epoch 21, Loss: 0.6315
Epoch 22, Loss: 0.6181
Epoch 23, Loss: 0.6037
Epoch 24, Loss: 0.5919
Epoch 25, Loss: 0.5796
Epoch 26, Loss: 0.5692
Epoch 27, Loss: 0.5588
Epoch 28, Loss: 0.5496
Epoch 29, Loss: 0.5406
Epoch 30, Loss: 0.5326
Epoch 31, Loss: 0.5248
Epoch 32, Loss: 0.5177
Epoch 33, Loss: 0.5108
Epoch 34, Loss: 0.5045
Epoch 35, Loss: 0.4985
Epoch 36, Loss: 0.4928
Epoch 37, Loss: 0.4874
Epoch 38, Loss: 0.4823
Epoch 39, Loss: 0.4775
Epoch 40, Loss: 0.4728
Epoch 41, Loss: 0.4684
Epoch 42, Loss: 0.4641
Epoch 43, Loss: 0.460

In [ ]:
label_op = []
input = x_test.iloc[24]
for model in model_net:
    prediction = model.predict(input)
    label_op.append(prediction)
max = 0
res = label_op[0]
for i in label_op:
    freq = label_op.count(i)
    if freq > max:
        max = freq
        res = i
print(label_op)        
print(res)

[array([4]), array([4]), array([9]), array([4]), array([4])]
[4]


In [ ]:
y_test[24]

4

In [ ]:
import scipy.io as sio

hyp_data = sio.loadmat("/content/drive/MyDrive/Colab Notebooks/SalinasA_corrected.mat")
ground_truth = sio.loadmat("/content/drive/MyDrive/Colab Notebooks/SalinasA_gt.mat")

In [ ]:
sorted(hyp_data.keys())

['__globals__', '__header__', '__version__', 'salinasA_corrected']

In [ ]:
x = hyp_data['salinasA_corrected']

In [ ]:
x.shape

(83, 86, 204)

In [ ]:
sorted(ground_truth.keys())

['__globals__', '__header__', '__version__', 'salinasA_gt']

In [ ]:
y = ground_truth['salinasA_gt']
print(y)

[[ 1  1  1 ... 12 12 12]
 [ 1  1  1 ... 12 12 12]
 [ 1  1  1 ... 12 12 12]
 ...
 [14 14 14 ...  0  0  0]
 [14 14 14 ...  0  0  0]
 [14 14 14 ...  0  0  0]]


In [ ]:
y.shape

(83, 86)

In [ ]:
y = y.ravel()

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 0], dtype=uint8)

In [ ]:
print(np.unique(y))

[ 0  1 10 11 12 13 14]


In [ ]:
pines_data = sio.loadmat("/content/drive/MyDrive/Colab Notebooks/Indian_pines_corrected.mat")
pines_gt = sio.loadmat("/content/drive/MyDrive/Colab Notebooks/Indian_pines_gt.mat")

In [ ]:
sorted(pines_data.keys())

['__globals__', '__header__', '__version__', 'indian_pines_corrected']

In [ ]:
sorted(pines_gt.keys())

['__globals__', '__header__', '__version__', 'indian_pines_gt']

In [ ]:
X = pines_data['indian_pines_corrected']

In [ ]:
Y = pines_gt['indian_pines_gt']

In [ ]:
Y = Y.ravel()
print(np.unique(Y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]


In [ ]:
X = X.reshape(X.shape[0]*X.shape[1],X.shape[2])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,random_state =42)

In [ ]:
nn = NeuralNetwork(X_train.shape[1],128,17)
nn.train(X_train,y_train,10,0.001)

Epoch 0, Loss: 16.7846
Epoch 1, Loss: 9.0209
Epoch 2, Loss: 16.2413
Epoch 3, Loss: 2.8330
Epoch 4, Loss: 2.8328
Epoch 5, Loss: 2.8326
Epoch 6, Loss: 2.8323
Epoch 7, Loss: 2.8321
Epoch 8, Loss: 2.8319
Epoch 9, Loss: 2.8316
